In [219]:
##   MLP: https://www.kaggle.com/rgaddati/santander-customer-satisfaction/santander1

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#from __future__ import division

import multiprocessing
N_PROCESSES = 8

%matplotlib inline  
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cross_validation import StratifiedKFold 
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import describe
from sklearn.preprocessing import StandardScaler, scale

In [220]:
df_train = pd.read_csv("Data/train.csv")
df_test = pd.read_csv("Data/test.csv")

print(df_test.shape)
print(df_train.shape)

(75818, 370)
(76020, 371)


In [221]:
## ## Join train and test data ## ##

# zero_col = np.zeros(df_test.shape[0])
# zero_col = pd.DataFrame(zero_col,columns=['id'])
# zero_col.iloc[:] = -1
# print(zero_col.shape)

# df_test["TARGET"] = zero_col
# print (df_test.shape)

# df_train = pd.concat ([df_train,df_test])
# df_train.shape

In [222]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

print(df_test.shape)
print(df_train.shape)

(75818, 336)
(76020, 337)


In [223]:
df_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,39205.170000,0
1,3,2,34,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49278.030000,0
2,4,2,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67333.770000,0
3,8,2,37,0,195,195,0,0,0,0,...,0,0,0,0,0,0,0,0,64007.970000,0
4,10,2,39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,117310.979016,0


In [224]:
# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

feature_names = df_train.columns.values 
feature_names = feature_names[1:feature_names.shape[0]-1]  ## removing ID and Target

print(df_test.shape)
print(df_train.shape)

(75818, 307)
(76020, 308)


In [225]:
## ## removing features with high presence of nulls

col_remove = X_train.shape[0]-sum(X_train == 0)<10
X_train = X_train[:,~col_remove]
X_test = X_test[:,~col_remove]
feature_names = feature_names[~col_remove]


print(X_train.shape)
print(X_test.shape)

(76020, 252)
(75818, 252)


In [226]:
train_size = X_train.shape[0]
print train_size

Data_all = np.concatenate([X_train,X_test],axis = 0)
Data_all.shape
print Data_all.shape

76020
(151838, 252)


In [227]:
##  I would suggest not to change -999999 into anything other  
## 
num_vals = (Data_all[:,0]==-999999)
print (sum(num_vals))
Data_all[num_vals,0] = -5


236


In [228]:
##  Converting cities to numbers
##

def get_cities_val_count (args):
    zero_col = args[0]
    cities_val = args[1]
    if sum(zero_col[:train_size] == cities_val)>60:
        new_col = np.zeros(zero_col.shape[0])
        new_col[(zero_col[:] == cities_val)] = 1
        return np.array(new_col)


zero_col = Data_all[:,0]
zero_col.shape

cities_vals = np.unique(Data_all[:train_size,0])

pool = multiprocessing.Pool(N_PROCESSES)
tt = pool.map(get_cities_val_count, zip([zero_col]*cities_vals.shape[0],cities_vals) )
pool.close()
pool.join()

tt = np.array(tt)
none_filter = map (lambda x: x is None, tt)
tt = tt[~np.array(none_filter)]
tt = np.vstack(tt).T

print Data_all.shape
Data_all = np.column_stack([Data_all, tt])  

print Data_all.shape
print "finished"

(151838, 252)
(151838, 268)
finished


In [229]:
## Get pandas dataframe with count for each val 
def get_vals(a):
    tt = map (lambda x: [x, sum(a==x)], np.unique(a))
    return pd.DataFrame(tt)  


aa = get_vals(X_train[:,138])
print aa.iloc[:,:]
pd.set_option("display.max_rows",255)
pd.set_option("display.max_colwidth",250)

               0      1
0  -1.000000e+00   1675
1  -9.038462e-01      1
2  -8.333333e-01      1
3  -8.000000e-01      1
4  -7.602877e-01      1
5  -6.666667e-01      1
6  -5.000000e-01      2
7  -4.995000e-01      1
8  -3.494898e-01      1
9  -3.333333e-01      2
10 -2.910826e-01      1
11 -1.973684e-01      1
12 -1.000000e-01      1
13 -9.090909e-02      1
14  0.000000e+00  73948
15  1.538462e-01      1
16  1.605039e-01      1
17  4.000000e-01      1
18  1.000000e+00      1
19  1.181818e+00      1
20  1.222222e+00      1
21  1.445714e+00      1
22  1.777778e+00      1
23  2.333092e+00      1
24  4.000000e+00      2
25  5.500000e+00      1
26  1.000000e+10    370


In [230]:
##  Counting number of zeros in columns
##


def count_values(a,val = 0):
    return sum(a==val)


# first select columns which are biased to zeroes:
pool = multiprocessing.Pool(N_PROCESSES)
tt = pool.map(count_values, map(lambda x: Data_all[:train_size,x],range(Data_all.shape[1])) )
pool.close()
pool.join()
tt = np.array(tt)

##  Count how many zeros are in this columns
sel_cols = (tt > Data_all[:train_size,:].shape[0]*0.990)
print "count columns: ", sum(sel_cols)


print (Data_all.shape)

pool = multiprocessing.Pool(N_PROCESSES)
tt = pool.map(count_values, map(lambda x: Data_all[x,sel_cols],range(Data_all.shape[0])) )
pool.close()
pool.join()

Data_all = np.column_stack((Data_all, tt)) 

## count zeros in all columns
pool = multiprocessing.Pool(N_PROCESSES)
tt = pool.map(count_values, map(lambda x: Data_all[x,:],range(Data_all.shape[0])) )
pool.close()
pool.join()

Data_all = np.column_stack((Data_all, tt)) 


print ("finished")


print (Data_all.shape)

count columns:  124
(151838, 268)
finished
(151838, 270)


In [218]:
def count_values(a,val = 9999999999):
    return sum(a==val)


# first select columns which are biased to zeroes:
pool = multiprocessing.Pool(N_PROCESSES)
tt = pool.map(count_values, map(lambda x: Data_all[:train_size,x],range(Data_all.shape[1])) )
pool.close()
pool.join()
tt = np.array(tt)

print tt
print sum(tt>0)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0 370  39   1  70  38  20
   5  42 370  39   1  70  42   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [217]:
feature_names[tt>0]

array(['delta_imp_aport_var13_1y3', 'delta_imp_aport_var17_1y3',
       'delta_imp_aport_var33_1y3', 'delta_imp_compra_var44_1y3',
       'delta_imp_reemb_var13_1y3', 'delta_imp_reemb_var17_1y3',
       'delta_imp_trasp_var33_in_1y3', 'delta_imp_venta_var44_1y3',
       'delta_num_aport_var13_1y3', 'delta_num_aport_var17_1y3',
       'delta_num_aport_var33_1y3', 'delta_num_compra_var44_1y3',
       'delta_num_venta_var44_1y3'], dtype=object)

In [ ]:
## replace missing values in columns with value that has the highest count
## get value with highest appearance rate 



def sort_by_unique_get_most_occur(arr_vals):
    t = dict()
    for val in np.unique(arr_vals): 
        t[val] = sum(arr_vals == val) 

    val0  = sorted (t, key = t.get, reverse = True)[0]
    return val0


for j in filter_cols_number:

    new_col = np.zeros(X_train.shape[0]) 
    new_col [X_train[:,j] == 9999999999.0] = 1
    X_train = np.column_stack((X_train, new_col))
    new_val = sort_by_unique_get_most_occur(X_train[X_train[:,j] != 9999999999.0,j])
    X_train[X_train[:,j] == 9999999999.0,j] = new_val

    new_col = np.zeros(X_test.shape[0])
    new_col [X_test[:,j] == 9999999999.0] = 1
    X_test = np.column_stack((X_test, new_col))     
    new_val = sort_by_unique_get_most_occur(X_test[X_test[:,j] != 9999999999.0,j])
    X_test[X_test[:,j] == 9999999999.0,j] = new_val

In [ ]:
## unique values in columns

pd.set_option("display.max_rows",255)
pd.set_option("display.max_colwidth",250)


tt = map(lambda x: np.unique(X_train[:,x]), range(X_train.shape[1]))
pd.DataFrame({'data': tt,'id': range(X_train.shape[1])},columns=['id','data'])

In [ ]:
print (np.unique(X_train[:,150]))
print (np.unique(X_test[:,150]))

In [ ]:
st_scale = StandardScaler(True, True, True)
X_train = st_scale.fit_transform(X_train)
X_test = st_scale.transform (X_test)

In [ ]:
sum(X_train[:,46] == -14.853708394243661)

In [ ]:
print (np.unique(X_train[:,150]))
print (np.unique(X_test[:,150]))

In [ ]:
#plt.plot(X_test.mean(axis=0), color ="b")
plt.plot(X_train.mean(axis=0), color ="r")
plt.show()
#X_test.mean(axis=0)

In [ ]:
#plt.plot(X_test.mean(axis=0), color ="b")
plt.plot(X_train.mean(axis=0), color ="r")
plt.show()
#X_test.mean(axis=0)

In [ ]:
## Feature selection

clf = ExtraTreesClassifier()
clf = clf.fit(X_train, y_train)
feature_selected = clf.feature_importances_

print ("ExtraTrees done")

feature_selected = np.argsort(-feature_selected)
print (feature_selected[0:30])

##   [305   1 268 270 266 264 263 221 265 267 223 150 143 269 219 218 128 224
##    152 220  22 233 222   2  11 189  71  13  14   4]

In [ ]:
#  https://www.kaggle.com/c/homesite-quote-conversion/forums/t/18837/what-is-your-best-single-model-score-and-how-did-you-achieve-it
#
#  Adding log features

feat = feature_selected[0:100]

c = X_train.shape[1]

k = 0

for i in feat:
    k = k+1
    if sum(X_train[:,i]<0)==0:
        new_col = np.log(X_train[:,i]+1)
        X_train = np.column_stack((X_train, new_col))

        new_col = np.sqrt(X_train[:,i])
        X_train = np.column_stack((X_train, new_col))

        new_col = 1/(0.001 + X_train[:,i])
        X_train = np.column_stack((X_train, new_col))        
        
        
        new_col = np.log(X_test[:,i]+1)
        X_test = np.column_stack((X_test, new_col))

        new_col = np.sqrt(X_test[:,i])
        X_test = np.column_stack((X_test, new_col))

        new_col = 1/(0.001 + X_test[:,i])
        X_test = np.column_stack((X_test, new_col))      
        
     
    
print(X_test.shape)
print(X_train.shape)

In [ ]:
#X_train.tofile("Data/proc_train.csv", sep=",",format="%.2f")
#X_test.tofile("Data/proc_test.csv", sep=",",format="%.2f")

np.savetxt("Data/proc_train.csv", X_train, delimiter=",",  fmt='%.3e')
np.savetxt("Data/proc_test.csv", X_test, delimiter=",",  fmt='%.3e')